In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-7.6999e-06), tensor(1.))

In [7]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [11]:
y_train[:20]

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9])

In [12]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [13]:
# num hidden
nh = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [14]:
torch.randn(m,nh)

tensor([[-1.4599,  0.4653, -1.4353,  ..., -2.3464,  0.4520, -0.8919],
        [ 0.6940,  1.5384, -0.4915,  ..., -1.0200,  0.4073, -1.2679],
        [-0.6050,  0.0290,  0.4150,  ..., -0.8220, -1.6744,  1.1132],
        ...,
        [ 1.2496,  0.4723,  2.5507,  ..., -0.6825, -1.5912,  0.8305],
        [ 0.9860, -0.1280, -1.4084,  ...,  0.7537,  0.1080, -1.2495],
        [ 0.5184, -1.1626,  1.3355,  ...,  0.6996, -0.4399, -0.5656]])

In [15]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [16]:
w1

tensor([[-0.0260, -0.0089, -0.0423,  ..., -0.0203, -0.0019, -0.0018],
        [ 0.0575, -0.0098,  0.0001,  ..., -0.0420,  0.0074, -0.0131],
        [-0.0269, -0.0406,  0.0224,  ..., -0.0458, -0.0484, -0.0283],
        ...,
        [ 0.0162, -0.0016,  0.0101,  ...,  0.0045, -0.0477,  0.0578],
        [-0.0006, -0.0585,  0.0202,  ..., -0.0468,  0.0190, -0.0174],
        [-0.0644, -0.0435, -0.0099,  ..., -0.0054, -0.0060, -0.0018]])

In [17]:
math.sqrt(m)

28.0

In [22]:
test_near_zero(w1.mean())
# w1.std = 1 -> 1/math.sqrt(m) because of kaiming init
test_near_zero(w1.std()-(1/math.sqrt(m)))

In [23]:
# This should be ~ (0,1) (mean,std)...
# because x_valid came from same distribution with x_train
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [24]:
def lin(x, w, b): return x@w + b

In [25]:
t = lin(x_valid, w1, b1)

In [26]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0660), tensor(0.9798))

In [27]:
def relu(x): return x.clamp_min(0.)

In [28]:
t = relu(lin(x_valid, w1, b1))

In [29]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.4189), tensor(0.5842))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [30]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [31]:
w1.mean(),w1.std()

(tensor(-1.4218e-05), tensor(0.0507))

In [32]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5834), tensor(0.8403))

In [33]:
#export
from torch.nn import init

In [36]:
w1 = torch.zeros(m,nh)
# fan_out because of tranpose weight of pytorch ref: torch.nn.functional.linear
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [37]:
# init.kaiming_normal_??

In [38]:
w1.mean(),w1.std()

(tensor(4.8362e-05), tensor(0.0505))

In [42]:
# min, std of output of lin before relu
lin(x_valid, w1, b1).mean(), lin(x_valid, w1, b1).std()

(tensor(-0.1002), tensor(1.4104))

In [43]:
t.mean(),t.std()

(tensor(0.5078), tensor(0.7953))

In [44]:
w1.shape

torch.Size([784, 50])

In [45]:
import torch.nn

In [46]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [48]:
# torch.nn.Linear.forward??

In [50]:
# torch.nn.functional.linear??

In [53]:
# torch.nn.Conv2d??

In [54]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [55]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [58]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
# fastai relu
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.1127), tensor(0.8799))

In [59]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [60]:
%timeit -n 10 _=model(x_valid)

6.06 ms ± 382 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [64]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [65]:
#export
def mse(output, targ): 
    return (output.squeeze(-1) - targ).pow(2).mean()

In [66]:
y_train,y_valid = y_train.float(),y_valid.float()

In [67]:
preds = model(x_train)

In [68]:
preds.shape

torch.Size([50000, 1])

In [69]:
mse(preds, y_train)

tensor(28.5194)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [72]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [73]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [74]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [75]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [76]:
forward_and_backward(x_train, y_train)

In [77]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [78]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [79]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [80]:
loss = forward(xt2, y_train)

In [81]:
loss.backward()

In [82]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [83]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [84]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [85]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [86]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [87]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [88]:
%time loss = model(x_train, y_train)

CPU times: user 156 ms, sys: 40 ms, total: 196 ms
Wall time: 48.2 ms


In [89]:
%time model.backward()

CPU times: user 4.01 s, sys: 7.42 s, total: 11.4 s
Wall time: 2.96 s


In [90]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [91]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [92]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [93]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [94]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [95]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [96]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [97]:
%time loss = model(x_train, y_train)

CPU times: user 140 ms, sys: 44 ms, total: 184 ms
Wall time: 46.1 ms


In [98]:
%time model.backward()

CPU times: user 280 ms, sys: 164 ms, total: 444 ms
Wall time: 111 ms


In [99]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [100]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [101]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [102]:
%time loss = model(x_train, y_train)

CPU times: user 144 ms, sys: 24 ms, total: 168 ms
Wall time: 41.7 ms


In [103]:
%time model.backward()

CPU times: user 264 ms, sys: 164 ms, total: 428 ms
Wall time: 107 ms


In [104]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [105]:
#export
from torch import nn

In [106]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [107]:
model = Model(m, nh, 1)

In [108]:
%time loss = model(x_train, y_train)

CPU times: user 120 ms, sys: 4 ms, total: 124 ms
Wall time: 35.7 ms


In [109]:
%time loss.backward()

CPU times: user 192 ms, sys: 20 ms, total: 212 ms
Wall time: 54 ms


## Export

In [110]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
